# 魔方自动复原

演示基于谷歌插件：[Rubiks Cube](https://chromewebstore.google.com/detail/%E8%B0%B7%E6%AD%8C%E6%B5%8F%E8%A7%88%E5%99%A8%E7%9A%84rubiks-cube/dlabgdldanmcjlmnifgogbnffionmfki)

### 安装依赖

In [ ]:
!pip install pyautogui
!pip install kociemba
!pip install opencv-python
!pip install imutils

# 一、实战

In [2]:
from rubik import *

## 1. 还原魔方

In [ ]:
# 魔方初始化，设置操作的时间间隔
cube = Cube(interval=0.2)

In [ ]:
# 识别并求解魔方
cube.auto_solve_cube(wait=False)

## 2. 构造给定魔方

In [15]:
txt = "DUDUUUDUDBRBRRRBRBLFLFFFLFLUDUDDDUDUFLFLLLFLFRBRBBBRBR"
state = input(f"请输入魔方状态的代码，直接回车使用示例代码\n示例：\n{txt}\n")
if not state: state = txt
try:
    kb.solve(state) # 检验是否有效
except:
    print("魔方代码输入有误！请检查")
    print(expand_cube(state))
print("输入魔方的展开图如下")
print(expand_cube(state))

cube = Cube()

input("按回车，开始构造给定魔方：")
sol = cube.to_cube_state(state)
print("魔方解法为")
for i in sol:
    print(i,end="\t")

请输入魔方状态的代码，直接回车使用示例代码
示例：
DUDUUUDUDBRBRRRBRBLFLFFFLFLUDUDDDUDUFLFLLLFLFRBRBBBRBR
 


输入魔方的展开图如下

          -----
        | D U D |
        | U U U |
        | D U D |
---------------------------------
| F L F | L F L | B R B | R B R |
| L L L | F F F | R R R | B B B |
| F L F | L F L | B R B | R B R |
---------------------------------
        | U D U |
        | D D D |
        | U D U |
          -----



按回车，开始构造给定魔方： 


将魔方化为给定状态，步数 22
魔方解法为
U'	R	U2	B'	U'	F	U	D'	B	D2	B'	L	U2	R2	U2	R2	D2	F2	R2	F2	D'	F2	

# 二、实现原理

## 1. 魔方检测

In [3]:
# 如果识别不正确，根据魔方大小修改 `min_scale, max_scale, num_step` 
scale_match.min_scale = 0.8 # 图像较小时调低，默认 0.8
scale_match.max_scale = 3.5 # 图像较大时调高，默认 3.0
scale_match.num_step = 80 # 匹配细度，默认 50 步

In [16]:
# 初始化魔方
cube = Cube(interval=0.15)

# 检查识别状态
print("注意：初始化时要隐藏下方截图，避免干扰图像检测")
cube.show_detection(openwindow=False)

注意：初始化时要隐藏下方截图，避免干扰图像检测


In [5]:
# 检查魔方中心位置
cube.check_center()

检查鼠标是否移动到中心位置


In [6]:
# 检查魔方小面位置
cube.check_facets(sides=[0,1,2]) # 顶面，左面，右面

正在检查顶面，注意查看鼠标位置是否准确
正在检查左侧面，注意查看鼠标位置是否准确
正在检查右侧面，注意查看鼠标位置是否准确


**显示匹配过程**：需安装 `ffmpeg`

In [7]:
from rubik.video import *

In [8]:
# Get frames
images = detect_image(cube.image, template, frames=True)
# images.sort(key = lambda x: x[0])
frames = [cv2PIL(img[1]) for img in images]

# export video
vid_name, vid_comp = "video.mp4", "video_comp.mp4"
imgs_to_video(frames, vid_name, fps=10)
display_video(vid_name, vid_comp)

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


## 2. 控制电脑操作

In [9]:
# 检查基本旋转
cube.check_basic_moves(facets="UDLRFB")

当前正在旋转的面为 U
当前正在旋转的面为 D
当前正在旋转的面为 L
当前正在旋转的面为 R
当前正在旋转的面为 F
当前正在旋转的面为 B


In [10]:
# 换心公式
cube.shift_center()
time.sleep(0.3)
cube.shift_center(back=True)

In [11]:
# 检查分布
state = cube.get_cube_distribution(string_code=True)

## 3. 颜色识别

In [13]:
# 检查颜色
# state = cube.get_cube_distribution(string_code=True)
print(expand_cube(state))


          -----
        | U D D |
        | R U D |
        | F L F |
---------------------------------
| L F U | L B D | R F F | L B B |
| U L B | R F L | F R U | F B B |
| U L B | D U R | U U L | D R F |
---------------------------------
        | R R B |
        | D D L |
        | R D B |
          -----



# 三、魔方群 

### 关于魔方群
1. **广义魔方群**：
    $$
    (C_2^{12}\rtimes S_{12}) \times (C_3^{8}\rtimes S_{8})
    $$
 
    注：(棱块群) 直积 (角块群)

2. **魔方群**：六个基本旋转生成，广义魔方群的真子群。

    注：实际上只用四个旋转就能生成魔方群了
   
3. 上帝之数 = 20

In [17]:
from math import factorial
# 魔方阶数
2**12 * factorial(12) * 3 ** 8 * factorial(8)

519024039293878272000

### 构造魔方状态

1. Kociemba 算法（2-phase algorithm）

2. 广义魔方群 <=> 魔方状态
   $$
   \begin{align*}
   Magic\ Group\ G&\rightarrow\ \{states\ of \ Rubik's\ Cube\}\\
   f&\mapsto f(I)
   \end{align*}
   $$

3. 魔方还原，本质上是求 状态 $S$ 的可逆元 $f$，并将 $f$ 写成基本旋转的组合。
   $$
   S \Rightarrow f(S) = I \Rightarrow f\cdot S = I
   $$

4. 借助群作用，扩展 Kociemba 算法的应用范围
   $$
   \begin{align*}
   &f_1\cdot S_{current} = I\\
   &f_2\cdot S_{target} = I\\
   &\Rightarrow S_{target} = f_2^{-1} = f_2^{-1}\cdot f_1\cdot S_{current}\\
   &\Rightarrow f_2^{-1}\cdot f_1 S_{mix} = I,\ where\ S_{mix} = f_1^{-1}\cdot f_2
   \end{align*}
   $$

简言之：将状态 $S_{current}\Rightarrow S_{target}$ 转化为普通的魔方还原问题 $S_{mix} \Rightarrow I$.